In [9]:
!pip install caer canaro

In [10]:
import os
import caer 
import canaro
import numpy as np
import cv2 as cv 
import gc

In [11]:
IMG_SIZE = (80,80)
channels = 1 
char_path = r'../input/the-simpsons-characters-dataset/simpsons_dataset'

In [12]:
char_dict = {}
for char in os.listdir(char_path):
    #save number of images in each folder there are 43 folder
    char_dict[char] = len(os.listdir(os.path.join(char_path,char)))

char_dict = caer.sort_dict(char_dict, descending=True)
char_dict

In [13]:
#get first 10 characters
characters = []
for i in char_dict:
    characters.append(i[0])
    if len(characters) >= 10 :
        break

characters

In [14]:
#Get all images from our characters
train = caer.preprocess_from_dir(char_path,characters,channels=channels,IMG_SIZE=IMG_SIZE,isShuffle=True)


In [15]:
len(train)

In [16]:
import matplotlib.pyplot as plt
plt.figure(figsize=(30,30))
plt.imshow(train[1][0],cmap='gray')
plt.show()

In [17]:
#split train into features and labels 
featureSet , labels = caer.sep_train(train,IMG_SIZE=IMG_SIZE)

# Normalize data

In [18]:
from tensorflow.keras.utils import to_categorical
featureSet = caer.normalize(featureSet)
labels = to_categorical(labels,len(characters))

# Split data

In [19]:
x_train , x_val , y_train , y_val = caer.train_val_split(featureSet,labels,val_ratio=0.2)

In [20]:
#delete unused data
del train 
del featureSet
del labels
gc.collect()

# Image Data Generator from existing images (help model to perform better)


In [21]:
BATCH_SIZE = 32
EPOCHS = 10

In [22]:
datagen = canaro.generators.imageDataGenerator()
train_gen = datagen.flow(x_train,y_train, batch_size = BATCH_SIZE)

# Model 

In [23]:
model = canaro.models.createSimpsonsModel(IMG_SIZE = IMG_SIZE , channels=channels , output_dim=len(characters),loss='binary_crossentropy',decay=1e-6,learning_rate=0.001,momentum=0.9,nesterov=True)

In [24]:
model.summary()

In [25]:
from tensorflow.keras.callbacks import LearningRateScheduler
callbacks_list = (LearningRateScheduler(canaro.lr_schedule))

In [26]:
training = model.fit(train_gen,
                    steps_per_epoch=len(x_train)//BATCH_SIZE,
                    epochs=EPOCHS,
                    validation_data=(x_val,y_val),
                    validation_steps=len(y_val)//BATCH_SIZE,
                    callbacks = callbacks_list)

In [27]:
test_path = r'../input/the-simpsons-characters-dataset/kaggle_simpson_testset/kaggle_simpson_testset/charles_montgomery_burns_0.jpg'
img = cv.imread(test_path)
plt.imshow(img,cmap='gray')
plt.show()

In [30]:
def prepare_test_data(img):
    img = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    img = cv.resize(img,IMG_SIZE)
    img = caer.reshape(img,IMG_SIZE,1)
    return img

In [31]:
predictions = model.predict(prepare_test_data(img))

In [32]:
predictions

In [34]:
print(characters[np.argmax(predictions[0])])
test_path = r'../input/the-simpsons-characters-dataset/kaggle_simpson_testset/kaggle_simpson_testset/charles_montgomery_burns_0.jpg'
img = cv.imread(test_path)
plt.imshow(img,cmap='gray')
plt.show()